In [25]:
import json
from babyTLSUtil import *
from random import randint
from os import urandom
from key_exchange import *
from cipher_suites import *
import telnetlib
import json
from random import choice

# The cipher suite negotiation assumes that the server does not change the cipher suite to the client
# ----- CIPHER SUITE NEGOTIATION -----
TEMP_CIPHER_SUITE = choice(list(CIPHER_SUITES.keys()))
TEMP_KEY_EXCHANGE_ALG = TEMP_CIPHER_SUITE.split("_")[1]
TEMP_KEY_EXCHANGE_STRUCT = choice(list(CIPHER_SUITES[TEMP_CIPHER_SUITE]["KEY_EXCHANGE"].keys()))
client_hello, KEP = generate_client_hello(TEMP_KEY_EXCHANGE_ALG, TEMP_KEY_EXCHANGE_STRUCT, CIPHER_SUITES, TEMP_CIPHER_SUITE)
print("Client Hello:", client_hello) # send to server

HOST = "0.0.0.0"
PORT = 1337

tn = telnetlib.Telnet(HOST, PORT)


def readline():
    return tn.read_until(b"\n")

def json_recv():
    line = readline()
    return json.loads(line.decode())

def json_send(hsh):
    request = json.dumps(hsh).encode()
    tn.write(request)


print(readline())
json_send(client_hello)
server_hello = json.loads(readline().decode()[38:])
print("Server hello:", server_hello)
server_public_key = server_hello["Key_Share"]
shared_secret = KEP.generate_shared_secret(server_public_key)

encrypt_and_digest = CIPHER_SUITES[TEMP_CIPHER_SUITE]["AUTHENTICATED_CIPHER"][0]
decrypt_and_verify = CIPHER_SUITES[TEMP_CIPHER_SUITE]["AUTHENTICATED_CIPHER"][1]

# hello verification
server_hello = json.dumps(server_hello)
HASH_ALG = CIPHER_SUITES[TEMP_CIPHER_SUITE]["HANDSHAKE_AUTHENTICATION"][0]
HASH_LEN = CIPHER_SUITES[TEMP_CIPHER_SUITE]["HANDSHAKE_AUTHENTICATION"][1]
result = hello_verify([client_hello, server_hello], shared_secret, HASH_ALG, HASH_LEN)
print(result)

server_certificate = json.loads(readline().decode()[20:-1])
server_certificate_verify = json.loads(readline().decode()[27:-1])
finished = json.loads(readline().decode()[27:-1])

# Master key calculation (encrypts application traffic)
key = result["server_handshake_key"]
print(key)
iv = result["server_handshake_iv"]

cert = decrypt_and_verify(server_certificate["Encrypted_Data"], server_certificate["Record_Header"], key, iv, server_certificate["Auth_Tag"])
print(cert)
signature = decrypt_and_verify(server_certificate_verify["Encrypted_Data"], server_certificate_verify["Record_Header"], key, iv, server_certificate_verify["Auth_Tag"])
# TODO: verify "verify_data"
verify_data = decrypt_and_verify(finished["Encrypted_Data"], finished["Record_Header"], key, iv, finished["Auth_Tag"])
handshake_hash = HASH_ALG(client_hello.encode() + server_hello.encode() + cert + signature + verify_data).digest()
last_result = handshake_verify(handshake_hash, bytes.fromhex(result["handshake_secret"]), HASH_ALG, HASH_LEN)
print(last_result)

# Client Handshake Finished
# finished_key = HKDF-Expand-Label(key: client_secret, label: "finished", ctx: "", len: 32)
finished_key = HKDF_expand_label(bytes.fromhex(result["client_secret"]), b"finished", b"", 32, HASH_ALG)
# finished_hash = SHA384(Client Hello ... Server Finished)
finished_hash = HASH_ALG(client_hello.encode() + server_hello.encode() + cert + signature + verify_data).digest()
# verify_data = HMAC-SHA384(key: finished_key, msg: finished_hash)
client_verify_data = tls_HMAC(finished_key, finished_hash, HASH_ALG)
header = "1703030045" # Record Header - CHANGE THIS: depends on the data follows
key = result["client_handshake_key"]
iv = result["client_handshake_iv"]
client_finished_encrypted, auth_tag = encrypt_and_digest(client_verify_data, header, key, iv)
client_finished = {
    "Record_Header": header,
    "Encrypted_Data": client_finished_encrypted.hex(),
    "Auth_Tag": auth_tag.hex()
}
json_send(client_finished)
print(client_finished)

Client Hello: {"Random": "25533c20128aa05874bd7ceff7f79466", "Cipher_Suites": ["TLS_ECDHE_RSA_PSS_SHA256_AES_256_GCM_SHA384", "TLS_DHE_RSA_PSS_SHA256_AES_256_GCM_SHA384", "TLS_ECDHE_RSA_PSS_SHA256_CHACHA20_POLY1305_SHA256", "TLS_DHE_RSA_PSS_SHA256_CHACHA20_POLY1305_SHA256"], "Temporary_Cipher_Suite": "TLS_DHE_RSA_PSS_SHA256_AES_256_GCM_SHA384", "Key_Share": 18949589637427817738059723251289625806243687466672248926150779220115036864101024104554481585262422399015712017981951033826326523397641837173193776578908660132941990057624374017799635757772017127847826847622899530053308159256112151434025785267743648174253154575487597182238644765927241669889458989250680128133548566869081323459602607497025170363547803606304497231223936761941221192933173942593228834035179010103116960408217826082405500311597669934013552354948079394965170639537139860446332451851840767178082352114243864160764690766348446544875542590133205569495999070604681909208145655428329420664810693967905287377956, "Key_Share_Struct": "